In [2]:
import math
import heapq
import numpy as np

In [3]:
test_items = np.load('Data/test_items.npy', allow_pickle=True)
test_users = np.load('Data/test_users.npy', allow_pickle=True)
topK = 10

In [4]:
def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

In [5]:
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

In [6]:
def eval_one_rating(idx, model_predictions):
    user = test_users[idx][0]
    items = test_items[idx]
    predictions = model_predictions[idx]
    gtItem = items[-1]
    
    map_item_score = {}
    
    for i in range(len(items)):
        item = items[i]
        map_item_score[item] = predictions[i]

    # Evaluate top rank list
    ranklist = heapq.nlargest(topK, map_item_score, key=map_item_score.get)
    hr = getHitRatio(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    return (hr, ndcg)

In [7]:
def evaluate_model(model_predictions):
    hits, ndcgs = [], []
    for idx in range(len(test_items)):
        (hr, ndcg) = eval_one_rating(idx, model_predictions)
        hits.append(hr)
        ndcgs.append(ndcg)
    
    hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print('Eval: HR = %.4f, NDCG = %.4f' % (hr, ndcg))
    return hr, ndcg

In [8]:
model_predictions = np.load('Predictions/mlp.npy', allow_pickle=True)
evaluate_model(model_predictions)

Eval: HR = 0.5705, NDCG = 0.3107


(0.5705196182396607, 0.31070297061217683)

In [9]:
model_predictions = np.load('Predictions/seq_i.npy', allow_pickle=True)
evaluate_model(model_predictions)

Eval: HR = 0.1962, NDCG = 0.1008


(0.19618239660657477, 0.10077311188637267)

In [10]:
model_predictions = np.load('Predictions/seq_iu.npy', allow_pickle=True)
evaluate_model(model_predictions)

Eval: HR = 0.1845, NDCG = 0.0928


(0.18451749734888653, 0.0928493924719518)

In [20]:
model_predictions = np.load('Predictions/bpr.npy', allow_pickle=True)
display(model_predictions.shape)
evaluate_model(model_predictions)

(943, 101)

Eval: HR = 0.6214, NDCG = 0.3461


(0.6214209968186638, 0.3461055973477752)

In [23]:
for k_size in [2, 5, 10, 20, 30, 50, 100]:
    print('K =', k_size)
    model_predictions = np.load('Predictions/kmeans_' + str(k_size) + '.npy', allow_pickle=True)
    evaluate_model(model_predictions)
    print('--------------')

K = 2
Eval: HR = 0.1516, NDCG = 0.0663
--------------
K = 5
Eval: HR = 0.1357, NDCG = 0.0552
--------------
K = 10
Eval: HR = 0.1135, NDCG = 0.0415
--------------
K = 20
Eval: HR = 0.1463, NDCG = 0.0587
--------------
K = 30
Eval: HR = 0.1379, NDCG = 0.0538
--------------
K = 50
Eval: HR = 0.1315, NDCG = 0.0502
--------------
K = 100
Eval: HR = 0.1273, NDCG = 0.0458
--------------
